In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
import os
from enum import Enum

pd.set_option("display.precision", 5)
pd.set_option('display.max_rows', None)

In [2]:
metadata_file = "./results/metadata.pkl"

class dataset_types(Enum):
    train = 1
    development = 2
    test = 3
    
    def title(self):
        return self._shorten_names[self.value - 1]

    def __lt__(self, other):
        return self.value < other.value
dataset_types._shorten_names = ["Train", "Dev", "Test"]

def save_results(y_pred, index, name, task, lenguage, dataset_type, group=None, description=None, truth=False, filename=None):
    
    path = f"./results/{task}/{lenguage}/{dataset_type.name}{'/' + group if group is not None else ''}/{name if filename is None else filename}.pkl"
    
    directory = "/".join(path.split("/")[:-1])
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    if os.path.exists(metadata_file):
        metadata = pd.read_pickle(metadata_file)
    else:
        metadata = pd.DataFrame({
            "Path": pd.Series([], dtype=str),
            "Name": pd.Series([], dtype=str),
            "Description": pd.Series([], dtype=str),
            "Dataset type": pd.Categorical([], categories=dataset_types, ordered=False),
            "Groud Truth": pd.Series([], dtype=bool),
            "Group": pd.Series([], dtype=str),
            "Task": pd.Series([], dtype=str),
            "Lenguage": pd.Series([], dtype=str),
        }).set_index("Path")
    
    if path in metadata.index:
        metadata = remove_results(path)

    metadata.loc[path] = {"Name": name, "Description": description, "Dataset type": dataset_type, "Groud Truth": truth, "Group": group, "Task": task, "Lenguage": lenguage}
    results = pd.DataFrame({"id": index, "y_pred": y_pred}).set_index("id") 
    
    results.to_pickle(path)
    metadata.to_pickle(metadata_file)
    
    print("Results saved on: " + path)

def remove_results(path=None):
    if os.path.exists(metadata_file):
        metadata = pd.read_pickle(metadata_file)
        if path is not None:
            if os.path.exists(path):
                metadata = metadata.drop(path)
                os.remove(path)
                metadata.to_pickle(metadata_file)
        else:
            if os.path.exists(metadata_file):
                used_files = [os.path.normpath(f) for f in metadata.index]
                all_files = set([os.path.normpath(os.path.join(dp, f)) for dp, dn, filenames in os.walk('./results') for f in filenames][1:])
                for f in used_files:
                    if f not in all_files:
                        os.remove(f)
        return metadata

    
def load_results():
    if os.path.exists(metadata_file):
        metadata = pd.read_pickle(metadata_file)
    else:
        return None
    
    metadata["Results"] = [pd.read_pickle(path) for path in metadata.index]
    return metadata

In [3]:
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score

def print_score(y_true, y_pred, name, f1_average):
    classification_report_results = acc = f1 = precision = recall =  None
    if y_true is not None:
        classification_report_results = classification_report(y_true, y_pred)

        acc, f1 = accuracy_score(y_true, y_pred), f1_score(y_true, y_pred, average=f1_average)   
        precision, recall = precision_score(y_true, y_pred), recall_score(y_true, y_pred)

    print(name)
    print('F1 macro: ', f1)
    print('Accuracy: ', acc)

    print('\nClassification Report')
    print('======================================================')
    print('\n', classification_report_results)
    
    return {"F1": f1, "Accuracy": acc, "Precision": precision, "Recall": recall}

def print_score_hateval_task1(y_true, y_pred, name):
    return print_score(y_true, y_pred, name, 'macro')
    
def print_score_detoxis_task1(y_true, y_pred, name):
    return print_score(y_true, y_pred, name, 'binary')

In [4]:
df_results = load_results()

mask = df_results["Groud Truth"] == True

df_truth = df_results[mask]
df_pred = df_results[~mask]

df_truth.sort_values(by=["Lenguage", "Dataset type"])

,Name,Description,Dataset type,Groud Truth,Group,Task,Lenguage,Results
Path,,,,,,,,
./results/hateval2019/task1/english/train/train_truth_task1.pkl,English Train,None,dataset_types.train,True,None,hateval2019/task1,english,y_pred id 201 1 202 ...
./results/hateval2019/task1/english/development/dev_truth_task1.pkl,English Development,None,dataset_types.development,True,None,hateval2019/task1,english,y_pred id 18201 0 1820...
./results/hateval2019/task1/english/test/test_truth_task1.pkl,English Test,None,dataset_types.test,True,None,hateval2019/task1,english,y_pred id 34243 0 3059...
./results/hateval2019/task1/spanish/train/train_truth_task1.pkl,Spanish Train,None,dataset_types.train,True,None,hateval2019/task1,spanish,y_pred id 20001 1 2000...
./results/detoxis/task1/spanish/train/train_truth_task1.pkl,Detoxis Train,None,dataset_types.train,True,None,detoxis/task1,spanish,y_pred id 0 0 1 ...
./results/hateval2019/task1/spanish/development/dev_truth_task1.pkl,Spanish Development,None,dataset_types.development,True,None,hateval2019/task1,spanish,y_pred id 20005 0 2000...
./results/detoxis/task1/spanish/development/dev_truth_task1.pkl,Detoxis Development,None,dataset_types.development,True,None,detoxis/task1,spanish,y_pred id 0 0 1 ...
./results/hateval2019/task1/spanish/test/test_truth_task1.pkl,Spanish Test,None,dataset_types.test,True,None,hateval2019/task1,spanish,y_pred id 31494 0 3246...


In [5]:
results = []

for path, (name, desc, dataset_type, truth, group, task, lenguage, y_pred) in df_pred.iterrows():
    y_true = df_truth[(df_truth["Dataset type"] == dataset_type) & (df_truth["Task"] == task) & (df_truth["Lenguage"] == lenguage)]["Results"]
    y_true = [None] if y_true.empty else y_true
    
    result = {}
    if task == "hateval2019/task1":
        result = print_score_hateval_task1(y_true[0], y_pred, name)
    if task == "detoxis/task1":
        result = print_score_detoxis_task1(y_true[0], y_pred, name)
        
    result.update({"Dataset type": dataset_type.title(),
                   "Task": ' '.join(task.split('/')).title(),
                   "Name": name,
                   "Group": ' '.join(group.split('_')).title(),
                   "Lenguage": lenguage.title(),
                   "Description": desc})
    
    results.append(result)

Bert base
F1 macro:  0.9299943995519642
Accuracy:  0.9315555555555556

Classification Report

               precision    recall  f1-score   support

           0       0.95      0.93      0.94      5217
           1       0.91      0.93      0.92      3783

    accuracy                           0.93      9000
   macro avg       0.93      0.93      0.93      9000
weighted avg       0.93      0.93      0.93      9000

Bert base
F1 macro:  0.7469041129594169
Accuracy:  0.749

Classification Report

               precision    recall  f1-score   support

           0       0.81      0.73      0.77       573
           1       0.68      0.77      0.72       427

    accuracy                           0.75      1000
   macro avg       0.75      0.75      0.75      1000
weighted avg       0.76      0.75      0.75      1000

Bert base
F1 macro:  0.5941497231031642
Accuracy:  0.6053333333333333

Classification Report

               precision    recall  f1-score   support

           0       

Bert Avarage
F1 macro:  0.9804141018466703
Accuracy:  0.9873646209386282

Classification Report

               precision    recall  f1-score   support

           0       0.99      0.99      0.99      1869
           1       0.99      0.97      0.98       901

    accuracy                           0.99      2770
   macro avg       0.99      0.98      0.99      2770
weighted avg       0.99      0.99      0.99      2770

Bert Avarage
F1 macro:  0.6481876332622601
Accuracy:  0.7619047619047619

Classification Report

               precision    recall  f1-score   support

           0       0.80      0.84      0.82       447
           1       0.68      0.62      0.65       246

    accuracy                           0.76       693
   macro avg       0.74      0.73      0.73       693
weighted avg       0.76      0.76      0.76       693

Bert Avarage
F1 macro:  None
Accuracy:  None

Classification Report

 None
Atalaya
F1 macro:  0.3424908424908425
Accuracy:  0.7407942238267148

Classi

Support Vector Classification
F1 macro:  0.5599999999999999
Accuracy:  0.7619047619047619

Classification Report

               precision    recall  f1-score   support

           0       0.75      0.95      0.84       447
           1       0.81      0.43      0.56       246

    accuracy                           0.76       693
   macro avg       0.78      0.69      0.70       693
weighted avg       0.77      0.76      0.74       693

Support Vector Classification (best)
F1 macro:  0.5972222222222222
Accuracy:  0.7489177489177489

Classification Report

               precision    recall  f1-score   support

           0       0.77      0.87      0.82       447
           1       0.69      0.52      0.60       246

    accuracy                           0.75       693
   macro avg       0.73      0.70      0.71       693
weighted avg       0.74      0.75      0.74       693

AdaBoost classifier
F1 macro:  0.5057471264367815
Accuracy:  0.6897546897546898

Classification Report

     

C:\Users\Ferran\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ferran\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ferran\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ferran\

Random Forest classifier (best)
F1 macro:  0.9977190608856654
Accuracy:  0.9977777777777778

Classification Report

               precision    recall  f1-score   support

           0       1.00      1.00      1.00      5217
           1       1.00      1.00      1.00      3783

    accuracy                           1.00      9000
   macro avg       1.00      1.00      1.00      9000
weighted avg       1.00      1.00      1.00      9000

Random Forest classifier
F1 macro:  0.9989737648560049
Accuracy:  0.999

Classification Report

               precision    recall  f1-score   support

           0       1.00      1.00      1.00      5217
           1       1.00      1.00      1.00      3783

    accuracy                           1.00      9000
   macro avg       1.00      1.00      1.00      9000
weighted avg       1.00      1.00      1.00      9000

Support Vector Classification (best)
F1 macro:  0.9540676206890405
Accuracy:  0.9553333333333334

Classification Report

           

...

Bernoulli Naive Bayes classifier
F1 macro:  0.21604938271604937
Accuracy:  0.6334776334776335

Classification Report

               precision    recall  f1-score   support

           0       0.66      0.90      0.76       447
           1       0.45      0.14      0.22       246

    accuracy                           0.63       693
   macro avg       0.55      0.52      0.49       693
weighted avg       0.58      0.63      0.57       693

Bernoulli Naive Bayes classifier (best)
F1 macro:  0.3614457831325302
Accuracy:  0.5411255411255411

Classification Report

               precision    recall  f1-score   support

           0       0.65      0.64      0.64       447
           1       0.36      0.37      0.36       246

    accuracy                           0.54       693
   macro avg       0.50      0.50      0.50       693
weighted avg       0.54      0.54      0.54       693

Ridge Classifier
F1 macro:  0.26038781163434904
Accuracy:  0.6147186147186147

Classification Report



MLP SMOBD
F1 macro:  0.9523287671232876
Accuracy:  0.9685920577617328

Classification Report

               precision    recall  f1-score   support

           0       0.98      0.97      0.98      1869
           1       0.94      0.96      0.95       901

    accuracy                           0.97      2770
   macro avg       0.96      0.97      0.96      2770
weighted avg       0.97      0.97      0.97      2770

MLP G_SMOTE
F1 macro:  0.9238521836506158
Accuracy:  0.9509025270758122

Classification Report

               precision    recall  f1-score   support

           0       0.96      0.97      0.96      1869
           1       0.93      0.92      0.92       901

    accuracy                           0.95      2770
   macro avg       0.95      0.94      0.94      2770
weighted avg       0.95      0.95      0.95      2770

MLP CCR
F1 macro:  0.11921891058581706
Accuracy:  0.6906137184115524

Classification Report

               precision    recall  f1-score   support

     

SVC SMOBD
F1 macro:  0.9052268811028144
Accuracy:  0.9404332129963899

Classification Report

               precision    recall  f1-score   support

           0       0.94      0.97      0.96      1869
           1       0.94      0.87      0.91       901

    accuracy                           0.94      2770
   macro avg       0.94      0.92      0.93      2770
weighted avg       0.94      0.94      0.94      2770

SVC G_SMOTE
F1 macro:  0.9073114565342544
Accuracy:  0.9418772563176895

Classification Report

               precision    recall  f1-score   support

           0       0.94      0.97      0.96      1869
           1       0.94      0.87      0.91       901

    accuracy                           0.94      2770
   macro avg       0.94      0.92      0.93      2770
weighted avg       0.94      0.94      0.94      2770

SVC CCR
F1 macro:  0.04125950054288817
Accuracy:  0.6812274368231047

Classification Report

               precision    recall  f1-score   support

     

Ridge G_SMOTE
F1 macro:  0.694331983805668
Accuracy:  0.7819494584837545

Classification Report

               precision    recall  f1-score   support

           0       0.87      0.79      0.83      1869
           1       0.64      0.76      0.69       901

    accuracy                           0.78      2770
   macro avg       0.76      0.78      0.76      2770
weighted avg       0.80      0.78      0.79      2770

Ridge CCR
F1 macro:  0.39089481946624804
Accuracy:  0.71985559566787

Classification Report

               precision    recall  f1-score   support

           0       0.73      0.93      0.82      1869
           1       0.67      0.28      0.39       901

    accuracy                           0.72      2770
   macro avg       0.70      0.61      0.60      2770
weighted avg       0.71      0.72      0.68      2770

Ridge LVQ_SMOTE
F1 macro:  0.6485849056603773
Accuracy:  0.7848375451263538

Classification Report

               precision    recall  f1-score   support

Bert Avarage (2 epochs| differnt lr)
F1 macro:  0.6325581395348837
Accuracy:  0.772005772005772

Classification Report

               precision    recall  f1-score   support

           0       0.78      0.89      0.83       447
           1       0.74      0.55      0.63       246

    accuracy                           0.77       693
   macro avg       0.76      0.72      0.73       693
weighted avg       0.77      0.77      0.76       693

Bert Avarage (2 epochs| differnt lr)
F1 macro:  None
Accuracy:  None

Classification Report

 None


In [6]:
df_results = pd.DataFrame(results).set_index(["Task", "Lenguage", "Dataset type", "Group", "Name", "Description"]).sort_index(level=[0, 1, 2, 3, 4])
scores = df_results.columns.to_list()
df_results_index = df_results.sort_values(by=["Task", "Lenguage", "Dataset type"] + scores, ascending=3*[True] + len(scores) * [False]).droplevel("Group")
df_results_index.droplevel("Description")

F1  \
Task              Lenguage Dataset type Name                                                 
Detoxis Task1     Spanish  Dev          Bert base (2 epochs)                       0.66368   
                                        Ridge SMOBD                                0.66038   
                                        MLP SMOTE_TomekLinks                       0.66019   
                                        Ridge G_SMOTE                              0.65672   
                                        Bert base (3 epochs)                       0.65471   
                                        Ridge SMOTE_TomekLinks                     0.65291   
                                        Ridge SMOTE_IPF                            0.65028   
                                        Bert Avarage                               0.64819   
                                        Ridge Assembled_SMOTE                      0.64540   
                                        Bert base (4 epochs)                       0.64069   
                                        Ridge polynom_fit_SMOTE                    0.64060   
                                        MLP Assembled_SMOTE                        0.63315   
                                        Bert Avarage (2 epochs| differnt lr)       0.63256   
                                        MLP SMOBD                                  0.62979   
                                        Bert Avarage (3 epochs)                    0.62882   
                                        SVC SMOBD                                  0.62559   
                                        MLP LVQ_SMOTE                              0.62343   
                                        Ridge ProWSyn                              0.62197   
                                        SVC G_SMOTE                                0.62053   
                                        MLP SMOTE_IPF                              0.61983   
                                        Ridge LVQ_SMOTE                            0.61915   
                                        SVC SMOTE_IPF                              0.61611   
                                        MLP G_SMOTE                                0.61339   
                                        SVC ProWSyn                                0.61205   
                                        Multi-layer Perceptron classifier          0.61072   
                                        SVC Assembled_SMOTE                        0.60766   
                                        SVC SMOTE_TomekLinks                       0.60432   
                                        MLP ProWSyn                                0.60041   
                                        MLP polynom_fit_SMOTE                      0.60000   
                                        Support Vector Classification (best)       0.59722   
                                        Ridge Classifier                           0.57985   
                                        SVC LVQ_SMOTE                              0.57963   
                                        Ridge Classifier (best)                    0.57957   
                                        SVC polynom_fit_SMOTE                      0.56383   
                                        Support Vector Classification              0.56000   
                                        Multi-layer Perceptron classifier (best)   0.55208   
                                        AdaBoost classifier (best)                 0.54505   
                                        Dummy Classifier (best)                    0.52396   
                                        AdaBoost classifier (best)                 0.52081   
                                        AdaBoost classifier                        0.50575   
                                        Ridge CCR                                  0.38527   
                                        Random

In [7]:
df_results_columns = df_results.groupby(["Task", "Lenguage",'Group','Name'], as_index=False).aggregate([(lambda t: lambda x: x[:, :, t] if x.index.isin([t], level=2).any() else None)(t.title()) for t in dataset_types])
df_results_columns.columns = pd.MultiIndex.from_product([scores, [t.title() for t in dataset_types]])
df_results_columns = df_results_columns.sort_values(by=['Task', 'Lenguage', 'Group', ("F1", dataset_types.test.title()), ("F1", dataset_types.development.title())], ascending=[True, True, True, False, False])

df_results_columns

F1  \
                                                                                           Train   
Task              Lenguage Group              Name                                                 
Detoxis Task1     Spanish  Deep Learning      Bert base (2 epochs)                       0.83636   
                                              Bert base (3 epochs)                       0.85899   
                                              Bert Avarage                               0.98041   
                                              Bert base (4 epochs)                       0.95833   
                                              Bert Avarage (2 epochs| differnt lr)       0.86307   
                                              Bert Avarage (3 epochs)                    0.94378   
                                              Atalaya                                    0.34249   
                           Sbert              Multi-layer Perceptron classifier          0.65392   
                                              Ridge Classifier                           0.62939   
                                              Support Vector Classification              0.82315   
                                              AdaBoost classifier                        0.64030   
                                              Random Forest classifier                   0.99944   
                           Sbert Best         Support Vector Classification (best)       0.99778   
                                              Ridge Classifier (best)                    0.65409   
                                              Multi-layer Perceptron classifier (best)   0.54848   
                                              AdaBoost classifier (best)                 0.88953   
                                              Random Forest classifier (best)            0.86962   
                           Sbert Oversampling Ridge SMOBD                                0.69273   
                                              MLP SMOTE_TomekLinks                       0.74348   
                                              Ridge G_SMOTE                              0.69433   
                                              Ridge SMOTE_TomekLinks                     0.68725   
                                              Ridge SMOTE_IPF                            0.68886   
                                              Ridge Assembled_SMOTE                      0.69830   
                                              Ridge polynom_fit_SMOTE                    0.69402   
                                              MLP Assembled_SMOTE                        0.72934   
                                              MLP SMOBD                                  0.95233   
                                              SVC SMOBD                                  0.90523   
                                              MLP LVQ_SMOTE                              0.69575   
                                              Ridge ProWSyn                              0.69336   
                                              SVC G_SMOTE                                0.90731   
                                              MLP SMOTE_IPF                              0.88106   
                                              Ridge LVQ_SMOTE                            0.64858   
                                              SVC SMOTE_IPF                              0.91327   
                                              MLP G_SMOTE                                0.92385   
                                              SVC ProWSyn                                0.90376   
                                              SVC Assembled_SMOTE                        0.91097   
                                              SVC SMOTE_TomekLinks                       0.90940   
                                              MLP ProWSyn                                0.926

In [8]:
df_results_columns2 = df_results_columns#.droplevel('Group')
df_results_columns2 = df_results_columns2.sort_values(by=["Task", "Lenguage", ("F1", dataset_types.test.title()), ("F1", dataset_types.development.title())], ascending=[True, True, False, False])
df_results_columns2

F1  \
                                                                                           Train   
Task              Lenguage Group              Name                                                 
Detoxis Task1     Spanish  Deep Learning      Bert base (2 epochs)                       0.83636   
                           Sbert Oversampling Ridge SMOBD                                0.69273   
                                              MLP SMOTE_TomekLinks                       0.74348   
                                              Ridge G_SMOTE                              0.69433   
                           Deep Learning      Bert base (3 epochs)                       0.85899   
                           Sbert Oversampling Ridge SMOTE_TomekLinks                     0.68725   
                                              Ridge SMOTE_IPF                            0.68886   
                           Deep Learning      Bert Avarage                               0.98041   
                           Sbert Oversampling Ridge Assembled_SMOTE                      0.69830   
                           Deep Learning      Bert base (4 epochs)                       0.95833   
                           Sbert Oversampling Ridge polynom_fit_SMOTE                    0.69402   
                                              MLP Assembled_SMOTE                        0.72934   
                           Deep Learning      Bert Avarage (2 epochs| differnt lr)       0.86307   
                           Sbert Oversampling MLP SMOBD                                  0.95233   
                           Deep Learning      Bert Avarage (3 epochs)                    0.94378   
                           Sbert Oversampling SVC SMOBD                                  0.90523   
                                              MLP LVQ_SMOTE                              0.69575   
                                              Ridge ProWSyn                              0.69336   
                                              SVC G_SMOTE                                0.90731   
                                              MLP SMOTE_IPF                              0.88106   
                                              Ridge LVQ_SMOTE                            0.64858   
                                              SVC SMOTE_IPF                              0.91327   
                                              MLP G_SMOTE                                0.92385   
                                              SVC ProWSyn                                0.90376   
                           Sbert              Multi-layer Perceptron classifier          0.65392   
                           Sbert Oversampling SVC Assembled_SMOTE                        0.91097   
                                              SVC SMOTE_TomekLinks                       0.90940   
                                              MLP ProWSyn                                0.92675   
                                              MLP polynom_fit_SMOTE                      0.91958   
                           Sbert Best         Support Vector Classification (best)       0.99778   
                           Sbert              Ridge Classifier                           0.62939   
                           Sbert Oversampling SVC LVQ_SMOTE                              0.84950   
                           Sbert Best         Ridge Classifier (best)                    0.65409   
                           Sbert Oversampling SVC polynom_fit_SMOTE                      0.86547   
                           Sbert              Support Vector Classification              0.82315   
                           Sbert Best         Multi-layer Perceptron classifier (best)   0.54848   
                                              AdaBoost classifier (best)                 0.88953   
                           Traditional Best   Dummy Classifier (best)                    0.490

In [9]:
df_results_columns.loc["Detoxis Task1"].reset_index(level=2).groupby(by=["Group"]).first()

Name       F1           \
                                                            Train      Dev   
Group                                                                        
Deep Learning                       Bert base (2 epochs)  0.83636  0.66368   
Sbert                  Multi-layer Perceptron classifier  0.65392  0.61072   
Sbert Best          Support Vector Classification (best)  0.99778  0.59722   
Sbert Oversampling                           Ridge SMOBD  0.69273  0.66038   
Traditional                             Ridge Classifier  0.82495  0.26039   
Traditional Best                 Dummy Classifier (best)  0.49087  0.52396   

                        Accuracy               Precision                \
                   Test    Train      Dev Test     Train      Dev Test   
Group                                                                    
Deep Learning       NaN  0.89928  0.78355  NaN   0.88682  0.74000  NaN   
Sbert               NaN  0.79747  0.75902  NaN   0.73611  0.71585  NaN   
Sbert Best          NaN  0.99856  0.74892  NaN   1.00000  0.69355  NaN   
Sbert Oversampling  NaN  0.77870  0.74026  NaN   0.63163  0.61620  NaN   
Traditional         NaN  0.90072  0.61472  NaN   0.96716  0.40870  NaN   
Traditional Best    NaN  0.32527  0.35498  NaN   0.32527  0.35498  NaN   

                     Recall                
                      Train      Dev Test  
Group                                      
Deep Learning       0.79134  0.60163  NaN  
Sbert               0.58824  0.53252  NaN  
Sbert Best          0.99556  0.52439  NaN  
Sbert Oversampling  0.76693  0.71138  NaN  
Traditional         0.71920  0.19106  NaN  
Traditional Best    1.00000  1.00000  NaN

In [10]:
df_results.loc[:, :, :, :, "Bert Avarage"]

F1  \
Task              Lenguage Dataset type Group         Description                                                   
Detoxis Task1     Spanish  Dev          Deep Learning Pretrained model with bert-base-multilingual-un...  0.64819   
                           Test         Deep Learning Pretrained model with bert-base-multilingual-un...      NaN   
                           Train        Deep Learning Pretrained model with bert-base-multilingual-un...  0.98041   
Hateval2019 Task1 English  Dev          Deep Learning Pretrained model with bert-base-uncased checkpo...  0.75085   
                           Test         Deep Learning Pretrained model with bert-base-uncased checkpo...  0.59399   
                           Train        Deep Learning Pretrained model with bert-base-uncased checkpo...  0.93938   
                  Spanish  Dev          Deep Learning Pretrained model with bert-base-multilingual-un...  0.83122   
                           Test         Deep Learning Pretrained model with bert-base-multilingual-un...  0.74574   
                           Train        Deep Learning Pretrained model with bert-base-multilingual-un...  0.98353   

                                                                                                          Accuracy  \
Task              Lenguage Dataset type Group         Description                                                    
Detoxis Task1     Spanish  Dev          Deep Learning Pretrained model with bert-base-multilingual-un...   0.76190   
                           Test         Deep Learning Pretrained model with bert-base-multilingual-un...       NaN   
                           Train        Deep Learning Pretrained model with bert-base-multilingual-un...   0.98736   
Hateval2019 Task1 English  Dev          Deep Learning Pretrained model with bert-base-uncased checkpo...   0.75300   
                           Test         Deep Learning Pretrained model with bert-base-uncased checkpo...   0.60600   
                           Train        Deep Learning Pretrained model with bert-base-uncased checkpo...   0.94078   
                  Spanish  Dev          Deep Learning Pretrained model with bert-base-multilingual-un...   0.83200   
                           Test         Deep Learning Pretrained model with bert-base-multilingual-un...   0.74687   
                           Train        Deep Learning Pretrained model with bert-base-multilingual-un...   0.98400   

                                                                                                          Precision  \
Task              Lenguage Dataset type Group         Description                                                     
Detoxis Task1     Spanish  Dev          Deep Learning Pretrained model with bert-base-multilingual-un...    0.68161   
                           Test         Deep Learning Pretrained model with bert-base-multilingual-un...        NaN   
                           Train        Deep Learning Pretrained model with bert-base-multilingual-un...    0.98871   
Hateval2019 Task1 English  Dev          Deep Learning Pretrained model with bert-base-uncased checkpo...    0.68750   
                           Test         Deep Learning Pretrained model with bert-base-uncased checkpo...    0.51729   
                           Train        Deep Learning Pretrained model with bert-base-uncased checkpo...    0.92186   
                  Spanish  Dev          Deep Learning Pretrained model with bert-base-multilingual-un...    0.78279   
                           Test         Deep Learning Pretrained model with bert-base-multilingual-un...    0.65306   
                           Train        Deep Learning Pretrained model with bert-base-multilingual-un...    0.97347   

                                                                                                           Recall  
Task              Lenguage Dataset type Group         Description                                 

Improvements:
- Allow removal of all files with specific features or codition (not just path)